In [1]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import GHMCMove, MCMCSampler
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
import numpy as np

In [12]:
from openmmtools import testsystems, alchemy
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False,
            disable_alchemical_dispersion_correction=True,
            alchemical_pme_treatment='direct-space')
test = testsystems.AlanineDipeptideVacuum()
alanine_vacuum = testsystems.AlanineDipeptideVacuum().system
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(22))
alanine_alchemical_system = factory.create_alchemical_system(reference_system=alanine_vacuum,
                                                           alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(alanine_alchemical_system)

In [13]:
from simtk import openmm, unit
thermodynamic_state = ThermodynamicState(system=alanine_alchemical_system, temperature=300*unit.kelvin)
compound_state = CompoundThermodynamicState(thermodynamic_state=thermodynamic_state, composable_states=[alchemical_state])
print(compound_state.lambda_sterics)
print(compound_state.lambda_electrostatics)

1.0
1.0


In [15]:
sampler_state = SamplerState(test.positions)

In [16]:
ghmc_move = GHMCMove(timestep=1.0*unit.femtosecond, n_steps=50)

In [17]:
sampler = MCMCSampler(compound_state, sampler_state, move=ghmc_move)

In [18]:
sampler.minimize()
sampler.run(n_iterations=2)

In [32]:
np.allclose(sampler.sampler_state.positions, test.positions)

False